In [8]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정.
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
num_users, num_items = R.shape
K=3

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [5]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [6]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장.
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트.
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  0.016410959588961445
### iteration step :  50  rmse :  0.01637308966218993
### iteration step :  100  rmse :  0.016333882336881784
### iteration step :  150  rmse :  0.016293661274599174
### iteration step :  200  rmse :  0.01625272572302336
### iteration step :  250  rmse :  0.016211338984681307
### iteration step :  300  rmse :  0.016169725523756753
### iteration step :  350  rmse :  0.016128072491623933
### iteration step :  400  rmse :  0.016086533303745463
### iteration step :  450  rmse :  0.016045231961723017
### iteration step :  500  rmse :  0.016004267423347165
### iteration step :  550  rmse :  0.015963717671935662
### iteration step :  600  rmse :  0.01592364333325902
### iteration step :  650  rmse :  0.015884090797210332
### iteration step :  700  rmse :  0.01584509485896164
### iteration step :  750  rmse :  0.015806680922219037
### iteration step :  800  rmse :  0.01576886681829548
### iteration step :  850  rmse :  0.015731664296432853


In [7]:
pred_matrix = np.dot(P, Q.T)
print('실제 행렬:\n', R)
print('\n예측 행렬:\n', np.round(pred_matrix, 3))

실제 행렬:
 [[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]

예측 행렬:
 [[3.991 1.171 1.266 2.    1.649]
 [6.288 4.978 0.896 2.983 1.003]
 [6.401 0.866 2.987 3.978 3.986]
 [4.969 2.005 1.006 2.013 1.258]]


원본 행렬과 비교해 널이 아닌 값은 큰 차이가 나지 않으며, 널인 값은 새로운 예측값으로 채워짐